In [8]:
import pandas as pd
import pyodbc as odbc


In [7]:
server = 'cmpcsb01'
database = 'conestoga'
username = 'remotequery'
password = 'excel'
cnxn = odbc.connect('DRIVER={SQL Server}; SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password, trusted_connection='no')
cursor = cnxn.cursor()
print(cursor)


## Call Program 52

In [9]:
query = '''
            execute spMovementAnalysis '20230517','20230517',
            -- meat Item Range:
            '100','999',
            -- Cost Center Start range:
            '500','1999',
            -- cost center end range:
            '400','499',
            -- Shift
            '1','2'
'''

df = pd.read_sql(query, cnxn)

df.head()

C:\Users\msmetanin\AppData\Local\Temp\ipykernel_19964\1590895568.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


,MovementDate,MovementDateFloat,ReferenceNumber,ItemKey,MovementType,PostingType,CostCenterKey1,NLotNo1,LotDate1,CostCenterKey2,CaseQty,Kgs,UserKey,TransactDateTime,OrderKey,AddressKey,SSCC,PalletNumber,TimeOfDay
0,2023-05-17,20230517.0,133136254277,602.0,Exit,Data Transfer,600.0,2.0,2023-05-16,451.0,0.0,585.0,593.0,2023-05-17 05:16:08,0.0,0.0,382928880512953247,0,05:16:08.0000000
1,2023-05-17,20230517.0,133136254259,111.0,Exit,Data Transfer,600.0,2.0,2023-05-16,416.0,1.0,889.0,514.0,2023-05-17 05:36:10,0.0,0.0,382928880512950307,0,05:36:10.0000000
2,2023-05-17,20230517.0,133136254178,309.0,Exit,Data Transfer,620.0,2.0,2023-05-16,481.0,1.0,1089.2,645.0,2023-05-17 05:43:48,0.0,0.0,382928880512895776,0,05:43:48.0000000
3,2023-05-17,20230517.0,133136254167,309.0,Exit,Data Transfer,620.0,2.0,2023-05-16,482.0,1.0,995.6,645.0,2023-05-17 05:44:04,0.0,0.0,82928880512886827,0,05:44:04.0000000
4,2023-05-17,20230517.0,163136178461,103.0,Exit,Data Transfer,600.0,1.0,2023-05-16,416.0,1.0,500.5,593.0,2023-05-17 05:59:45,0.0,0.0,82928880512856820,0,05:59:45.0000000


In [10]:
column_name = 'Kgs'
duplicates = df[df.duplicated(subset=column_name, keep=False)][['ItemKey', column_name]]
duplicates

,ItemKey,Kgs
3,309.0,995.6
4,103.0,500.5
9,501.0,981.6
30,603.0,762.0
45,811.0,569.8
...,...,...
634,603.0,736.0
636,808.0,630.6
640,815.0,681.2
641,766.0,1147.2


In [12]:
duplicates_count = duplicates.value_counts()

for value, count in duplicates_count.items():
    print(f" {value}: {count} times duplicated")

 (811.0, 15.0): 7 times duplicated
 (418.0, 12.0): 6 times duplicated
 (985.0, 0.0001): 2 times duplicated
 (715.0, 71.0): 2 times duplicated
 (715.0, 0.0001): 2 times duplicated
 (903.0, 0.0001): 2 times duplicated
 (904.0, 0.0001): 2 times duplicated
 (906.0, 0.0001): 2 times duplicated
 (907.0, 0.0001): 2 times duplicated
 (701.0, 0.0001): 2 times duplicated
 (901.0, 0.0001): 2 times duplicated
 (908.0, 0.0001): 2 times duplicated
 (603.0, 776.5): 2 times duplicated
 (603.0, 762.0): 2 times duplicated
 (912.0, 0.0001): 2 times duplicated
 (913.0, 0.0001): 2 times duplicated
 (828.0, 51.6): 2 times duplicated
 (902.0, 0.0001): 2 times duplicated
 (871.0, 323.2): 2 times duplicated
 (915.0, 0.0001): 2 times duplicated
 (766.0, 81.2): 2 times duplicated
 (766.0, 1147.2): 2 times duplicated
 (834.0, 309.1): 2 times duplicated
 (808.0, 630.6): 2 times duplicated
 (833.0, 294.2): 2 times duplicated
 (810.0, 896.2): 2 times duplicated
 (811.0, 569.8): 2 times duplicated
 (811.0, 947.6): 2 

In [20]:
duplicate_counts = df.groupby(['ItemKey', 'Kgs']).size().reset_index(name='Count')

duplicate_counts[(duplicate_counts['Count'] > 1) & (duplicate_counts['Kgs'] > 1)]


,ItemKey,Kgs,Count
10,103.0,500.5,2
27,154.0,804.2,2
28,154.0,861.4,2
30,154.0,868.2,2
34,154.0,914.0,2
37,154.0,987.0,2
123,412.0,442.0,2
164,418.0,12.0,6
172,421.0,10.0,2
204,501.0,999.0,2


In [26]:
df_grouped = df.groupby('ItemKey')['Kgs'].sum()

df_grouped.to_csv('Q:\Operations\Reports\summary.csv', header=True)
df_grouped

ItemKey
101.0     9955.5001
103.0     4579.4001
104.0     2431.4002
111.0     2769.9000
116.0        0.0001
            ...    
960.0     7200.0002
961.0     1336.0002
962.0    13024.9462
963.0     4595.4602
985.0        0.0002
Name: Kgs, Length: 123, dtype: float64